In [1]:
#Importing useful libs
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns',None)

In [2]:
#Reading data
data_path="D://ML/Kaggle/hk/"
data = pd.read_csv(data_path+"modeling_data.csv")
oot_data = pd.read_csv(data_path+"oot_data.csv")

In [3]:
#Defining target column as the (winner horse number -1) of the race
ALL_TARGET_COLS = [x for x in data.columns if 'result' in x]
data['Target']=data[ALL_TARGET_COLS].apply(lambda row: int(''.join([x for x in row[row == 1].index[0] if x.isdigit()]))-1, axis=1)
oot_data['Target']=oot_data[ALL_TARGET_COLS].apply(lambda row: int(''.join([x for x in row[row == 1].index[0] if x.isdigit()]))-1, axis=1)

In [4]:
data.head(3)

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,race_class,"('horse_age', 1)","('horse_country', 1)","('horse_type', 1)","('horse_rating', 1)","('declared_weight', 1)","('actual_weight', 1)","('draw', 1)","('win_odds', 1)","('place_odds', 1)","('trainer_id', 1)","('jockey_id', 1)","('horse_age', 2)","('horse_country', 2)","('horse_type', 2)","('horse_rating', 2)","('declared_weight', 2)","('actual_weight', 2)","('draw', 2)","('win_odds', 2)","('place_odds', 2)","('trainer_id', 2)","('jockey_id', 2)","('horse_age', 3)","('horse_country', 3)","('horse_type', 3)","('horse_rating', 3)","('declared_weight', 3)","('actual_weight', 3)","('draw', 3)","('win_odds', 3)","('place_odds', 3)","('trainer_id', 3)","('jockey_id', 3)","('horse_age', 4)","('horse_country', 4)","('horse_type', 4)","('horse_rating', 4)","('declared_weight', 4)","('actual_weight', 4)","('draw', 4)","('win_odds', 4)","('place_odds', 4)","('trainer_id', 4)","('jockey_id', 4)","('horse_age', 5)","('horse_country', 5)","('horse_type', 5)","('horse_rating', 5)","('declared_weight', 5)","('actual_weight', 5)","('draw', 5)","('win_odds', 5)","('place_odds', 5)","('trainer_id', 5)","('jockey_id', 5)","('horse_age', 6)","('horse_country', 6)","('horse_type', 6)","('horse_rating', 6)","('declared_weight', 6)","('actual_weight', 6)","('draw', 6)","('win_odds', 6)","('place_odds', 6)","('trainer_id', 6)","('jockey_id', 6)","('horse_age', 7)","('horse_country', 7)","('horse_type', 7)","('horse_rating', 7)","('declared_weight', 7)","('actual_weight', 7)","('draw', 7)","('win_odds', 7)","('place_odds', 7)","('trainer_id', 7)","('jockey_id', 7)","('horse_age', 8)","('horse_country', 8)","('horse_type', 8)","('horse_rating', 8)","('declared_weight', 8)","('actual_weight', 8)","('draw', 8)","('win_odds', 8)","('place_odds', 8)","('trainer_id', 8)","('jockey_id', 8)","('horse_age', 9)","('horse_country', 9)","('horse_type', 9)","('horse_rating', 9)","('declared_weight', 9)","('actual_weight', 9)","('draw', 9)","('win_odds', 9)","('place_odds', 9)","('trainer_id', 9)","('jockey_id', 9)","('horse_age', 10)","('horse_country', 10)","('horse_type', 10)","('horse_rating', 10)","('declared_weight', 10)","('actual_weight', 10)","('draw', 10)","('win_odds', 10)","('place_odds', 10)","('trainer_id', 10)","('jockey_id', 10)","('horse_age', 11)","('horse_country', 11)","('horse_type', 11)","('horse_rating', 11)","('declared_weight', 11)","('actual_weight', 11)","('draw', 11)","('win_odds', 11)","('place_odds', 11)","('trainer_id', 11)","('jockey_id', 11)","('horse_age', 12)","('horse_country', 12)","('horse_type', 12)","('horse_rating', 12)","('declared_weight', 12)","('actual_weight', 12)","('draw', 12)","('win_odds', 12)","('place_odds', 12)","('trainer_id', 12)","('jockey_id', 12)","('horse_age', 13)","('horse_country', 13)","('horse_type', 13)","('horse_rating', 13)","('declared_weight', 13)","('actual_weight', 13)","('draw', 13)","('win_odds', 13)","('place_odds', 13)","('trainer_id', 13)","('jockey_id', 13)","('horse_age', 14)","('horse_country', 14)","('horse_type', 14)","('horse_rating', 14)","('declared_weight', 14)","('actual_weight', 14)","('draw', 14)","('win_odds', 14)","('place_odds', 14)","('trainer_id', 14)","('jockey_id', 14)","('result', 1)","('result', 2)","('result', 3)","('result', 4)","('result', 5)","('result', 6)","('result', 7)","('result', 8)","('result', 9)","('result', 10)","('result', 11)","('result', 12)","('result', 13)","('result', 14)",Target
0,0,883.0,1,1,0,0,1400,2,14,485000.0,5,3.0,14.0,3.0,60.0,1089.0,120.0,1.0,5.4,1.7,128.0,138.0,3.0,1.0,3.0,60.0,1170.0,128.0,2.0,12.0,3.6,47.0,183.0,3.0,11.0,3.0,60.0,1135.0,123.0,3.0,8.6,2.5,4.0,155.0,3.0,1.0,3.0,60.0,1027.0,113.0,4.0,11.0,3.9,38.0,162.0,3.0,11.0,3.0,60.0,1114.0,127.0,5.0,7.0,1.8,54.0,34.0,3.0,11.0,3.0,60.0,1126.0,123.0,6.0,38.0,13.0,75.0,131.0,3.0,1.0,3.0,60.0,1020.0,133.0,7.0,9.7,3.7,118.0,2.0,3.0,11.0,3.0,60.0,1082.0,132.0,8.0,3.5,1.5,137.0,18.0,3.0,1.0,6.0,60.0,1072.0,125.

### Feature Selection

In [5]:
#Defining list of categorical columns which we will be passed as categorical variables in xgboost
cat_cols =['race_id','venue','config','surface','going','horse_ratings'] +\
[x for x in data.columns if 'horse_type' in x] +\
[x for x in data.columns if 'horse_country' in x] +\
[x for x in data.columns if 'trainer_id' in x] +\
[x for x in data.columns if 'jockey_id' in x]
TARGET_COLS = 'Target'

In [6]:
for col in cat_cols:
    data[col] = data[col].astype('category')

In [7]:
# create XGBoost instance with default hyper-parameters, high number of estimators
xgb_estimator = xgb.XGBClassifier(objective='multi:softprob', n_estimators=1000,max_depth=10,learning_rate=0.05,random_state=42,enable_categorical=True,tree_method="gpu_hist")
# fitting the model
xgb_estimator.fit(data[data.columns[:-15]], data[TARGET_COLS])

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=10, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [8]:
feature_importance = dict(zip(xgb_estimator.get_booster().feature_names, xgb_estimator.feature_importances_))
feature_importance=dict(sorted(feature_importance.items(),key=lambda x: x[1],reverse=True))

In [9]:
feature_importance

{'race_id': 0.085553765,
 "('jockey_id', 14)": 0.021302901,
 "('jockey_id', 13)": 0.017834948,
 "('win_odds', 2)": 0.017602853,
 "('win_odds', 3)": 0.01695591,
 "('win_odds', 1)": 0.015184092,
 "('win_odds', 5)": 0.012111081,
 "('win_odds', 4)": 0.011854552,
 "('trainer_id', 12)": 0.011809538,
 "('win_odds', 8)": 0.01149797,
 "('win_odds', 6)": 0.011447592,
 "('trainer_id', 11)": 0.010988008,
 "('win_odds', 7)": 0.010839063,
 "('jockey_id', 12)": 0.01063791,
 "('horse_type', 2)": 0.010629128,
 "('trainer_id', 10)": 0.010487086,
 "('jockey_id', 11)": 0.010433318,
 "('trainer_id', 9)": 0.010380691,
 "('jockey_id', 10)": 0.010375323,
 "('trainer_id', 2)": 0.010217737,
 "('win_odds', 9)": 0.010195766,
 "('trainer_id', 3)": 0.010134802,
 "('trainer_id', 7)": 0.010113363,
 "('trainer_id', 4)": 0.0100622,
 "('win_odds', 10)": 0.010059324,
 "('trainer_id', 1)": 0.009817186,
 "('jockey_id', 6)": 0.009767464,
 "('trainer_id', 5)": 0.009747579,
 "('horse_type', 9)": 0.009738459,
 "('trainer_id', 

In [10]:
top_vars = [x for x in feature_importance.keys() if feature_importance[x]>=0.004]
print(f"Initial features count: {len(feature_importance.items())}, Final features counts: {len(top_vars)}")

Initial features count: 165, Final features counts: 88


In [11]:
joblib.dump(top_vars, data_path+"pkl_files/top_vars.pkl")

['D://ML/Kaggle/hk/pkl_files/top_vars.pkl']

In [12]:
joblib.dump([x for x in cat_cols if x in top_vars], data_path+"pkl_files/cat_cols.pkl")

['D://ML/Kaggle/hk/pkl_files/cat_cols.pkl']

## Modeling and Cross Validation

In [13]:
N_FOLD = 5
TARGET_COLS='Target'
skf = StratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(skf.split(data, data[TARGET_COLS])):
    data.loc[val_index, 'fold'] = int(n)
data['fold'] = data['fold'].astype(int)
data['fold'].value_counts()

1    1255
2    1255
3    1255
0    1255
4    1254
Name: fold, dtype: int64

In [14]:
data.head(3)

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,race_class,"('horse_age', 1)","('horse_country', 1)","('horse_type', 1)","('horse_rating', 1)","('declared_weight', 1)","('actual_weight', 1)","('draw', 1)","('win_odds', 1)","('place_odds', 1)","('trainer_id', 1)","('jockey_id', 1)","('horse_age', 2)","('horse_country', 2)","('horse_type', 2)","('horse_rating', 2)","('declared_weight', 2)","('actual_weight', 2)","('draw', 2)","('win_odds', 2)","('place_odds', 2)","('trainer_id', 2)","('jockey_id', 2)","('horse_age', 3)","('horse_country', 3)","('horse_type', 3)","('horse_rating', 3)","('declared_weight', 3)","('actual_weight', 3)","('draw', 3)","('win_odds', 3)","('place_odds', 3)","('trainer_id', 3)","('jockey_id', 3)","('horse_age', 4)","('horse_country', 4)","('horse_type', 4)","('horse_rating', 4)","('declared_weight', 4)","('actual_weight', 4)","('draw', 4)","('win_odds', 4)","('place_odds', 4)","('trainer_id', 4)","('jockey_id', 4)","('horse_age', 5)","('horse_country', 5)","('horse_type', 5)","('horse_rating', 5)","('declared_weight', 5)","('actual_weight', 5)","('draw', 5)","('win_odds', 5)","('place_odds', 5)","('trainer_id', 5)","('jockey_id', 5)","('horse_age', 6)","('horse_country', 6)","('horse_type', 6)","('horse_rating', 6)","('declared_weight', 6)","('actual_weight', 6)","('draw', 6)","('win_odds', 6)","('place_odds', 6)","('trainer_id', 6)","('jockey_id', 6)","('horse_age', 7)","('horse_country', 7)","('horse_type', 7)","('horse_rating', 7)","('declared_weight', 7)","('actual_weight', 7)","('draw', 7)","('win_odds', 7)","('place_odds', 7)","('trainer_id', 7)","('jockey_id', 7)","('horse_age', 8)","('horse_country', 8)","('horse_type', 8)","('horse_rating', 8)","('declared_weight', 8)","('actual_weight', 8)","('draw', 8)","('win_odds', 8)","('place_odds', 8)","('trainer_id', 8)","('jockey_id', 8)","('horse_age', 9)","('horse_country', 9)","('horse_type', 9)","('horse_rating', 9)","('declared_weight', 9)","('actual_weight', 9)","('draw', 9)","('win_odds', 9)","('place_odds', 9)","('trainer_id', 9)","('jockey_id', 9)","('horse_age', 10)","('horse_country', 10)","('horse_type', 10)","('horse_rating', 10)","('declared_weight', 10)","('actual_weight', 10)","('draw', 10)","('win_odds', 10)","('place_odds', 10)","('trainer_id', 10)","('jockey_id', 10)","('horse_age', 11)","('horse_country', 11)","('horse_type', 11)","('horse_rating', 11)","('declared_weight', 11)","('actual_weight', 11)","('draw', 11)","('win_odds', 11)","('place_odds', 11)","('trainer_id', 11)","('jockey_id', 11)","('horse_age', 12)","('horse_country', 12)","('horse_type', 12)","('horse_rating', 12)","('declared_weight', 12)","('actual_weight', 12)","('draw', 12)","('win_odds', 12)","('place_odds', 12)","('trainer_id', 12)","('jockey_id', 12)","('horse_age', 13)","('horse_country', 13)","('horse_type', 13)","('horse_rating', 13)","('declared_weight', 13)","('actual_weight', 13)","('draw', 13)","('win_odds', 13)","('place_odds', 13)","('trainer_id', 13)","('jockey_id', 13)","('horse_age', 14)","('horse_country', 14)","('horse_type', 14)","('horse_rating', 14)","('declared_weight', 14)","('actual_weight', 14)","('draw', 14)","('win_odds', 14)","('place_odds', 14)","('trainer_id', 14)","('jockey_id', 14)","('result', 1)","('result', 2)","('result', 3)","('result', 4)","('result', 5)","('result', 6)","('result', 7)","('result', 8)","('result', 9)","('result', 10)","('result', 11)","('result', 12)","('result', 13)","('result', 14)",Target,fold
0,0,883.0,1,1,0,0,1400,2,14,485000.0,5,3.0,14.0,3.0,60.0,1089.0,120.0,1.0,5.4,1.7,128.0,138.0,3.0,1.0,3.0,60.0,1170.0,128.0,2.0,12.0,3.6,47.0,183.0,3.0,11.0,3.0,60.0,1135.0,123.0,3.0,8.6,2.5,4.0,155.0,3.0,1.0,3.0,60.0,1027.0,113.0,4.0,11.0,3.9,38.0,162.0,3.0,11.0,3.0,60.0,1114.0,127.0,5.0,7.0,1.8,54.0,34.0,3.0,11.0,3.0,60.0,1126.0,123.0,6.0,38.0,13.0,75.0,131.0,3.0,1.0,3.0,60.0,1020.0,133.0,7.0,9.7,3.7,118.0,2.0,3.0,11.0,3.0,60.0,1082.0,132.0,8.0,3.5,1.5,137.0,18.0,3.0,1.0,6.0,60.0,1072.0

In [15]:
data.columns[:-16]

Index(['race_id', 'date', 'venue', 'race_no', 'config', 'surface', 'distance',
       'going', 'horse_ratings', 'prize',
       ...
       '('horse_country', 14)', '('horse_type', 14)', '('horse_rating', 14)',
       '('declared_weight', 14)', '('actual_weight', 14)', '('draw', 14)',
       '('win_odds', 14)', '('place_odds', 14)', '('trainer_id', 14)',
       '('jockey_id', 14)'],
      dtype='object', length=165)

In [16]:
fold_models=dict()
scores=[]
#We create model which will predict if a particular horse will win or not
#5 fold training loop
for fold in range(N_FOLD):
    print(f"------------FOLD: {fold}------------")
    df_train = data[data['fold'] != fold].reset_index(drop=True)
    df_test = data[data['fold'] == fold].reset_index(drop=True)
    X_train, Y_train= df_train[top_vars], df_train[TARGET_COLS]
    X_test, Y_test= df_test[top_vars], df_test[TARGET_COLS]
    # create XGBoost instance with default hyper-parameters
    #Using multi:softprob because the result contains predicted probability of each data point belonging to each class as per official documentation
    xgb_estimator = xgb.XGBClassifier(objective='multi:softprob', 
                                      n_estimators = 100, 
                                      max_depth=10,
                                      reg_lambda=0.01,
                                      subsample=0.9,
                                      colsample_bytree=0.9,
                                      learning_rate=0.001,
                                      enable_categorical=True,
                                      tree_method="gpu_hist", 
                                      random_state=42,
                                      n_jobs=-1)
    # fit the model
    xgb_estimator.fit(X_train, Y_train)
    fold_models[f"model_fold_{fold}"] = xgb_estimator
    scores.append(accuracy_score(Y_test, xgb_estimator.predict(X_test)))
    print(classification_report(Y_test, xgb_estimator.predict(X_test)))
    print(f"Accuracy on fold {fold} is: {scores[-1]}")
print("\nAverage accuruacy of model",np.mean(scores, axis=0))

------------FOLD: 0------------
              precision    recall  f1-score   support

           0       0.35      0.40      0.37       131
           1       0.30      0.46      0.37       125
           2       0.35      0.38      0.36       123
           3       0.33      0.37      0.35       106
           4       0.31      0.40      0.35       111
           5       0.21      0.15      0.18        97
           6       0.20      0.21      0.20        92
           7       0.20      0.25      0.22        93
           8       0.31      0.30      0.31        87
           9       0.28      0.13      0.18        77
          10       0.38      0.18      0.24        73
          11       0.16      0.07      0.09        75
          12       0.31      0.28      0.30        32
          13       0.38      0.24      0.30        33

    accuracy                           0.29      1255
   macro avg       0.29      0.27      0.27      1255
weighted avg       0.29      0.29      0.28     

In [17]:
joblib.dump(fold_models, data_path+"pkl_files/fold_models.pkl")

['D://ML/Kaggle/hk/pkl_files/fold_models.pkl']